In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from mabwiser.mab import LearningPolicy

from alns import ALNS
from alns.accept import *
from alns.select import *
from alns.stop import *
import pandas as pd
import math
import os
import pyscipopt as scip

In [ ]:
%matplotlib inline

In [ ]:
SEED = 42

In [111]:
np.random.seed(SEED)

# Generic Problem State

In [78]:
class ProblemState:
    """
    Generic problem class for the mip problem. It stores the current
    solution as a vector of variables, one for each item.

    Current objective also stored.
    """
    def __init__(self, x, model):
        self.x=x #solution type:scip.sol
        self.model = model
        self.solution = self.transform_solution_to_array()#type: np.array
        self.incumbent = self.solution


    def objective(self):
        #retrieve the objective value of the current solution
        return self.model.getSolObjVal(self.x)

    def proxy_objective(self) -> int:
        p = np.random.randint(1, 100, size=len(self.solution))
        return p @ self.model

    def length(self) -> int:
        return len(self.solution)

    def get_solution(self):
        return self.solution

    def get_objective(self):
        # TODO output as minimize
        return self.model.getSolObjVal(self.x)

    #for now leave it empty
    def get_context(self):
        return None
    
    def array_to_sol(self):
        scip_sol = self.model.createSol()
        for var in self.model.getVars():
            self.model.setSolVal(scip_sol, var, self.solution[var]) #needs to be index
        return scip_sol
    def transform_solution_to_array(self):
        solution_array = []
        for i in range(self.model.getNVars()):
            solution_array.append(self.model.getSolVal(self.x, self.model.getVars()[i]))
        solution_array = np.array(solution_array)

        return solution_array


# MUTATION Destroy Operator mutation_op

In [92]:
def extract_variable_features(state:ProblemState):
        varbls = state.model.getVars()
        var_types = [v.vtype() for v in varbls]
        lbs = [v.getLbGlobal() for v in varbls]
        ubs = [v.getUbGlobal() for v in varbls]

        type_mapping = {"BINARY": 0, "INTEGER": 1, "IMPLINT": 2, "CONTINUOUS": 3}
        var_types_numeric = [type_mapping.get(t, 0) for t in var_types]

        variable_features = pd.DataFrame({
            'var_type': var_types_numeric,
            'var_lb': lbs,
            'var_ub': ubs
        })

        variable_features = variable_features.astype({'var_type': int, 'var_lb': float, 'var_ub': float})

        return variable_features  # pd.dataframe

def to_destroy_mut(discrete) -> int:
        delta = 0.25
        return int(delta * len(discrete))

def find_discrete(state:ProblemState):
        discrete = []
        for i in range(state.length()):
            var_features = extract_variable_features(state)
            if var_features['var_type'][i] == 0 or var_features['var_type'][i] == 1:
                discrete.append(i)
        return discrete

def mutation_op(state:ProblemState,rnd_state):

        discrete = find_discrete(state)

        to_remove = rnd_state.choice(discrete, size=to_destroy_mut(discrete))
        
        #fix variable yapman lazim burada 
        
    
        
        assignments = state.solution.copy()
        assignments[to_remove] = None 
        print(assignments)
        
        scip_sol = state.model.createSol()
        subMIP_vars = state.model.getVars()

        
        for i in range(state.model.getNVars()):
            val= assignments[i]
            state.model.setSolVal(scip_sol,subMIP_vars[i],val)
            
            
            
        #         for var in state.model.getVars()[var]:
        #             print("var",var)
        #             #state.model.setSolVal(scip_sol, var, assignments[var])


        return ProblemState(scip_sol,state.model)




# Mutation Repair

In [93]:
def repair_op(state: ProblemState,rnd_state) -> ProblemState:
    
    #Not the most effective way but it works!
    #for i in to_remove:
    #   state.model.addCons(state.model.getVars()[i] == state.solution[i])
    
    for var in state.model.getVars():
        if np.isnan(state.x[var]): 
            state.model.addCons(var == state.x[var])

            #model.addCons(x + y + z == 32, name="Heads")

            
    #s.getVal(x) == s.getSolVal(solution, x)
    state.model.optimize()


    solution = state.model.getBestSol()
    
    state = ProblemState(solution, state.model)
    #print(solution)
    return state


## ALNS

In [94]:
def make_alns() -> ALNS:
    rnd_state = np.random.RandomState(SEED)
    alns = ALNS(rnd_state)

    alns.add_destroy_operator(mutation_op)

    alns.add_repair_operator(repair_op)

    return alns

# Read MIP instance

In [95]:
import numpy as np
import pyscipopt as scip


class BaseRead:
    def __init__(self, problem_instance_file: str) -> None:
        self.model = scip.Model()
        self.model.hideOutput()
        self.model.readProblem(problem_instance_file)


class ReadInstance(BaseRead):
    def __init__(self, problem_instance_file: str) -> None:
        super().__init__(problem_instance_file)
        #self.var_features = self.extract_variable_features()


    def get_sense(self) -> str:
        """
        Returns
        -------
        objective -> Minimize or Maximize
        """

        sense = self.model.getObjectiveSense()
        return sense

    def get_model(self):
        return self.model


# Find initial feasible state (with solution gap and time)

In [96]:
def initial_state(instance_path,gap,time) -> ProblemState:
    # TODO implement a function that returns an initial solution

    # TODO Solve with scip stop at feasible
    instance = ReadInstance(problem_instance_file=instance_path)
    model = instance.get_model()

    # solution gap is less than %50  > STOP, terrible but, good start.
    model.setParam("limits/gap", gap)
    model.setParam('limits/time', time)
    model.optimize()
    solution = []
    for v in model.getVars():
        if v.name != "n":
            solution.append(model.getVal(v))
    #solution = np.array(solution)
    len_sol = len(solution)
    solution = model.getBestSol()
    #print("init sol", self.model.getObjVal())

    #solution2=model.createSol() #scip in icinde tanimli
    #solution3=model.createSol()
    state = ProblemState(solution, model)

    return state

# MIP instance from MIPLIB

In [97]:
instance_path = "neos-5140963-mincio.mps.gz"

In [98]:
# Terrible - but simple - two first solution, where only the first item is
# selected.
init_sol = initial_state(instance_path,0.50,30)

#print(init_sol.transform_solution_to_array())

init_sol2 = initial_state(instance_path,0.75,30)
print("Initial Feasible Solution:", init_sol2.transform_solution_to_array())


Initial Feasible Solution: [ 1.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0. 10. 11.  2.  1.  6.  5.  4.  3.  7.  9. 12.  8.  0.]


## Operator selection schemes

We now have everything set-up for solving the problem. We will now look at several of the operator selection schemes the `alns` package offers. The list is not exhaustive: for a complete overview, head over to `alns.select` in the documentation.

Here, we use the `HillClimbing` acceptance criterion, which only accepts better solutions.

In [99]:
accept = HillClimbing()

In [100]:
select = RouletteWheel(scores=[5, 2, 1, 0.5],
                       decay=0.8,
                       num_destroy=1,
                       num_repair=1)

alns = make_alns()
res = alns.iterate(init_sol, select, accept, MaxIterations(10))

print(f"Found solution with objective {res.best_state.objective()}.")

[ 0. nan  0.  1.  0.  0.  0.  0. nan  0.  0.  0.  0. nan nan  1.  0. nan
  0.  0. nan nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0. nan  1. nan  0.
 nan  0.  0. nan nan nan  0.  0.  0. nan  0.  0.  0.  0.  0.  0.  0. nan
 nan  0. nan  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. nan nan  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0.  0. nan nan  0.  0. nan nan
  0.  0.  0.  0.  0.  0.  0.  0. nan  1.  0.  0.  0. nan  0.  0.  0.  0.
  0.  1.  0. nan nan nan  0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  1.  0.  0. nan  0. nan  0.  0.  0.  0.  0. nan  0.  0. nan  1.
  0.  0.  0.  0. nan  0.  0. nan  0.  0.  0.  0. nan  0.  0.  0.  1.  0.
  0.  0.  0.  0. 11.  5.  6.  4.  3.  8.  7.  1. 12.  9.  2. 13.]
[ 0. nan  0. nan  0.  0.  0. nan nan  0.  0.  0.  0. nan nan  1.  0.  0.
  0.  0.  0.  0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0.
  0.  1.  0. nan nan  0.  0. nan nan  0.  0. nan  0. nan  

### More advanced bandit algorithms

Operator selection can be seen as a multi-armed-bandit problem.
Each operator pair is a bandit arm, and the reward for each arm corresponds to the evaluation outcome depending on the score array.
Accordingly, any multi-armed bandit algorithm can be used as an operator selection scheme.
ALNS integrates with [MABWiser](https://github.com/fidelity/mabwiser/) to provide access to more bandit algorithms. 
You may install MABWiser as an extra dependency using `pip install alns[mabwiser]`.

Here, we use a simple epsilon-greedy algorithm from MABWiser.
The algorithm picks a random operator pair with probability $\epsilon=0.15$ and otherwise chooses the operator pair with the largest mean so far.

In [101]:
select = MABSelector(scores=[5, 2, 1, 0.5],
                     num_destroy=1,
                     num_repair=1,
                     learning_policy=LearningPolicy.EpsilonGreedy(epsilon=0.15))

alns = make_alns()
res = alns.iterate(init_sol, select, accept, MaxIterations(10))

print(f"Found solution with objective {res.best_state.objective()}.")

[ 0. nan  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  1.  0. nan
  0.  0. nan nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0. nan  1. nan  0.
 nan  0.  0. nan nan  0.  0.  0.  0. nan  0.  0.  0.  0.  0.  0.  0. nan
 nan  0. nan  1.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0. nan nan  0.
  0.  0. nan  0.  0.  0.  0.  0.  0. nan  0.  0. nan nan  0.  0. nan nan
  0.  0.  0.  0.  0.  0.  0.  0. nan  1.  0.  0.  0. nan  0.  0.  0.  0.
  0.  1.  0. nan nan nan  0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.
  1.  0.  1.  0.  0. nan  0. nan  0.  0.  0.  0.  0. nan  0.  0. nan  1.
  0.  0.  0.  0. nan  0.  0. nan  0.  0.  0.  0. nan  0.  0.  0.  1. nan
  0.  0.  0.  0. 11.  5.  6.  4.  3.  8.  7.  1. 12.  9.  2. 13.]
[ 0. nan  0. nan  0.  0.  0. nan nan  0.  0.  0.  0. nan nan  1.  0.  0.
  0.  0.  0.  0.  0. nan  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. nan  0.
  0.  1.  0. nan nan  0.  0. nan nan  0.  0. nan  0. nan  